## Importing Libraries

In [1]:
import json
import requests
import pandas as pd
import numpy as np

## Importing Data From API and coverting it to DataFrame

In [2]:
r=requests.get(url="https://data.covid19india.org/v4/min/timeseries.min.json")
data=json.loads(r.content)
x=pd.DataFrame.transpose(pd.DataFrame(data))
x.reset_index(inplace=True)
x.head()

,index,dates
0,AN,"{'2020-03-26': {'delta': {'confirmed': 1}, 'de..."
1,AP,"{'2020-03-12': {'delta': {'confirmed': 1}, 'de..."
2,AR,"{'2020-04-02': {'delta': {'confirmed': 1}, 'de..."
3,AS,"{'2020-03-31': {'delta': {'confirmed': 1}, 'de..."
4,BR,"{'2020-03-22': {'delta': {'confirmed': 2}, 'de..."


## Extracting State and Date columns 

In [3]:
diii={}
for i in range(int(len(x))):
    key=(np.array(x)[i][0])
    value=list(dict(np.array(x)[i][1]).keys())
    
    diii[key]=value

stat=[]
dates=[]
for i,j in diii.items():
    for k in range(len(j)):
        stat.append(i)
        dates.append(j[k])
State_dates=pd.DataFrame({"State":stat,"dates":dates})
State_dates.head(10)

,State,dates
0,AN,2020-03-26
1,AN,2020-03-27
2,AN,2020-03-28
3,AN,2020-03-29
4,AN,2020-03-30
5,AN,2020-03-31
6,AN,2020-04-01
7,AN,2020-04-02
8,AN,2020-04-03
9,AN,2020-04-04


**Giving Sr.No to every entry to make unique records that will help for merging**

In [4]:
State_dates["Sr.No"]=np.arange(len(State_dates))

## For every Date Value Extracting delta delta7 total values and will join them with parent tabel (State_dates)

In [5]:
Concat_values=pd.DataFrame(x.loc[:,"dates"][0]).transpose()
for i in range(1,len(x.loc[:,"dates"])):
    first_st=pd.DataFrame(x.loc[:,"dates"][i]).transpose()
    Concat_values=pd.concat([Concat_values,first_st])
Concat_values.head(10)


,delta,delta7,total
2020-03-26,{'confirmed': 1},{'confirmed': 1},{'confirmed': 1}
2020-03-27,{'confirmed': 5},{'confirmed': 6},{'confirmed': 6}
2020-03-28,{'confirmed': 3},{'confirmed': 9},{'confirmed': 9}
2020-03-29,NaN,{'confirmed': 9},{'confirmed': 9}
2020-03-30,{'confirmed': 1},{'confirmed': 10},{'confirmed': 10}
2020-03-31,NaN,{'confirmed': 10},{'confirmed': 10}
2020-04-01,NaN,{'confirmed': 10},{'confirmed': 10}
2020-04-02,NaN,{'confirmed': 9},{'confirmed': 10}
2020-04-03,NaN,{'confirmed': 4},{'confirmed': 10}
2020-04-04,NaN,{'confirmed': 1},{'confirmed': 10}


In [6]:
## Adjusting Columns Of Concat_values and giving Sr.no to join them
Concat_values.reset_index(inplace=True)
Concat_values.columns=["Date","delta","delta7","total"]
Concat_values.insert(0,"Sr.No",np.arange(len(Concat_values)))
Concat_values.head(10)

,Sr.No,Date,delta,delta7,total
0,0,2020-03-26,{'confirmed': 1},{'confirmed': 1},{'confirmed': 1}
1,1,2020-03-27,{'confirmed': 5},{'confirmed': 6},{'confirmed': 6}
2,2,2020-03-28,{'confirmed': 3},{'confirmed': 9},{'confirmed': 9}
3,3,2020-03-29,NaN,{'confirmed': 9},{'confirmed': 9}
4,4,2020-03-30,{'confirmed': 1},{'confirmed': 10},{'confirmed': 10}
5,5,2020-03-31,NaN,{'confirmed': 10},{'confirmed': 10}
6,6,2020-04-01,NaN,{'confirmed': 10},{'confirmed': 10}
7,7,2020-04-02,NaN,{'confirmed': 9},{'confirmed': 10}
8,8,2020-04-03,NaN,{'confirmed': 4},{'confirmed': 10}
9,9,2020-04-04,NaN,{'confirmed': 1},{'confirmed': 10}


## From Extracted Data now we will further extract Delta and Other Values

In [7]:
##With Help of Key value of dictionaries taking Sr.NO and Delta sub Dictionaries and coverting them into dataFrame.
n=Concat_values.loc[:,["Sr.No","delta"]]
vv=np.array(n)
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]

In [8]:
delta=pd.DataFrame(DI).transpose()
delta.insert(0,"Sr.No",np.arange(len(delta)))

delta

,Sr.No,confirmed,recovered,tested,other,deceased,vaccinated1,vaccinated2
0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
21673,21673,976.0,837.0,43322.0,NaN,15.0,323292.0,229056.0
21674,21674,990.0,845.0,45437.0,NaN,9.0,532317.0,287998.0
21675,21675,982.0,860.0,49017.0,NaN,8.0,701727.0,317576.0
21676,21676,980.0,880.0,47131.0,NaN,13.0,756507.0,304433.0


**Merging Delta table with our Parent table , Sr.No being same columns**

In [9]:
Stat_delta=State_dates.merge(delta,on="Sr.No",how="inner")
Stat_delta.columns=['State', 'dates', 'Sr.No', 'deltaconfirmed', 'deltarecovered', 'deltatested', 'deltaother',
       'deltadeceased', 'deltavaccinated1', 'deltavaccinated2']

In [10]:
Stat_delta.head(10)

,State,dates,Sr.No,deltaconfirmed,deltarecovered,deltatested,deltaother,deltadeceased,deltavaccinated1,deltavaccinated2
0,AN,2020-03-26,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AN,2020-03-27,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AN,2020-03-28,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AN,2020-03-29,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AN,2020-03-30,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,AN,2020-03-31,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AN,2020-04-01,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AN,2020-04-02,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AN,2020-04-03,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AN,2020-04-04,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extracting Delta7

In [11]:
n=Concat_values.loc[:,["Sr.No","delta7"]]
vv=np.array(n)
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]
delta7=pd.DataFrame(DI).transpose()
delta7.columns=['delta7confirmed', 'delta7recovered', 'delta7tested', 'delta7other', 'delta7deceased', 'delta7vaccinated1',
       'delta7vaccinated2']

**Merging Delta7 table with our Extracted table , Sr.No being same columns**

In [12]:
delta7.insert(0,"Sr.No",np.arange(len(delta7)))
State_del_del7=Stat_delta.merge(delta7,on="Sr.No",how="inner")

## Extracting Total Column

In [13]:
n=Concat_values.loc[:,["Sr.No","total"]]
vv=np.array(n)
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]
total=pd.DataFrame(DI).transpose()
total.insert(0,"Sr.No",np.arange(len(total)))
total.columns=['Sr.No', 'totalconfirmed', 'totalrecovered', 'totaltested', 'totalother', 'totaldeceased',
       'totalvaccinated1', 'totalvaccinated2']

**Merging Delta7 table with our Extracted table , Sr.No being same columns. This Will be our Final Table**

In [14]:
final=State_del_del7.merge(total,how="inner",on="Sr.No")

In [15]:
final.head(10)

,State,dates,Sr.No,deltaconfirmed,deltarecovered,deltatested,deltaother,deltadeceased,deltavaccinated1,deltavaccinated2,...,delta7deceased,delta7vaccinated1,delta7vaccinated2,totalconfirmed,totalrecovered,totaltested,totalother,totaldeceased,totalvaccinated1,totalvaccinated2
0,AN,2020-03-26,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AN,2020-03-27,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AN,2020-03-28,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AN,2020-03-29,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
4,AN,2020-03-30,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
5,AN,2020-03-31,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
6,AN,2020-04-01,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
7,AN,2020-04-02,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
8,AN,2020-04-03,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
9,AN,2020-04-04,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#final.to_csv(r"C:\Users\91940\Downloads\time_final.csv")

## Extracting Statewise Data

In [39]:
r=requests.get(url="https://data.covid19india.org/v4/min/data.min.json")

In [40]:
data = json.loads(r.content)

In [41]:
#Coverting Data into DataFrame
tr=pd.DataFrame.transpose(pd.DataFrame(data))

In [42]:
#Geting State Names 
col1 = np.array(tr.index)
# Creating Saperate Column for State
tr["STATE"]=col1
tr.head(5)

,delta,delta21_14,delta7,districts,meta,total,STATE
AN,"{'tested': 1376, 'vaccinated1': 3, 'vaccinated...",{'confirmed': 9},"{'confirmed': 3, 'recovered': 5, 'tested': 893...","{'Nicobars': {'delta7': {'vaccinated1': 62, 'v...","{'date': '2021-10-31', 'last_updated': '2021-1...","{'confirmed': 7651, 'deceased': 129, 'recovere...",AN
AP,"{'confirmed': 385, 'deceased': 4, 'recovered':...",{'confirmed': 3220},"{'confirmed': 2873, 'deceased': 30, 'recovered...","{'Anantapur': {'delta': {'confirmed': 4, 'reco...","{'date': '2021-10-31', 'last_updated': '2021-1...","{'confirmed': 2066450, 'deceased': 14373, 'rec...",AP
AR,"{'confirmed': 1, 'recovered': 9, 'tested': 334...",{'confirmed': 87},"{'confirmed': 66, 'recovered': 97, 'tested': 4...","{'Anjaw': {'delta7': {'vaccinated1': 45, 'vacc...","{'date': '2021-10-31', 'last_updated': '2021-1...","{'confirmed': 55155, 'deceased': 280, 'recover...",AR
AS,"{'confirmed': 212, 'deceased': 1, 'recovered':...",{'confirmed': 1499},"{'confirmed': 2056, 'deceased': 24, 'recovered...","{'Baksa': {'delta': {'vaccinated1': 39, 'vacci...","{'date': '2021-10-31', 'last_updated': '2021-1...","{'confirmed': 610645, 'deceased': 5997, 'other...",AS
BR,"{'confirmed': 8, 'recovered': 9, 'tested': 226...",{'confirmed': 30},"{'confirmed': 40, 'recovered': 31, 'tested': 1...","{'Araria': {'delta': {'vaccinated1': 5177, 'va...","{'date': '2021-10-31', 'last_updated': '2021-1...","{'confirmed': 726098, 'deceased': 9661, 'other...",BR


In [43]:
# Shifting State Column to First Column
FIRST_COL=tr.pop("STATE")
tr.insert(0,"State",FIRST_COL)
#Convering Dataframe into array for Ease in Iteration (trail)
vv=np.array(tr.iloc[:,0:2])

In [44]:
# Trying to covert array into desired Dictionary (trail)
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]

In [45]:
vv[1][1]

{'confirmed': 385,
 'deceased': 4,
 'recovered': 675,
 'tested': 39848,
 'vaccinated1': 20497,
 'vaccinated2': 24137}

In [46]:
#Transforming data to get desired ouput (trial)
deltaa=pd.DataFrame(DI).transpose()
#inserting state at First Column
deltaa.insert(0,"State",FIRST_COL)
deltaa.head(5)

,State,tested,vaccinated1,vaccinated2,confirmed,deceased,recovered,other
AN,AN,1376.0,3.0,13.0,NaN,NaN,NaN,NaN
AP,AP,39848.0,20497.0,24137.0,385.0,4.0,675.0,NaN
AR,AR,334.0,42.0,195.0,1.0,NaN,9.0,NaN
AS,AS,15060.0,19124.0,37463.0,212.0,1.0,236.0,NaN
BR,BR,226443.0,114694.0,145827.0,8.0,NaN,9.0,NaN


In [47]:
#Changing Column Names
delta_columns=list(deltaa.columns)
delta_columns=['State','delta_tested','delta_vaccinated1','delta_vaccinated2','delta_confirmed','delta_deceased','delta_recovered','delta_other']
deltaa.columns=delta_columns
deltaa.reset_index(drop=True, inplace=True)
deltaa.head(5)

,State,delta_tested,delta_vaccinated1,delta_vaccinated2,delta_confirmed,delta_deceased,delta_recovered,delta_other
0,AN,1376.0,3.0,13.0,NaN,NaN,NaN,NaN
1,AP,39848.0,20497.0,24137.0,385.0,4.0,675.0,NaN
2,AR,334.0,42.0,195.0,1.0,NaN,9.0,NaN
3,AS,15060.0,19124.0,37463.0,212.0,1.0,236.0,NaN
4,BR,226443.0,114694.0,145827.0,8.0,NaN,9.0,NaN


In [48]:
vv=np.array(tr.iloc[:,0:2])
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]
deltaa=pd.DataFrame(DI).transpose()
deltaa.insert(0,"State",FIRST_COL)
deltaa.reset_index(drop=True, inplace=True)


In [49]:
# Query to Get State And delta21_14

vv=np.array(tr.loc[:,["State","delta21_14"]])
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]
delta21_14=pd.DataFrame(DI).transpose()
delta21_14.insert(0,"State",FIRST_COL)
delta21_14.reset_index(drop=True, inplace=True)
delta21_14.columns=["State","delta21_14_confirmed"]
delta21_14.head(5)

,State,delta21_14_confirmed
0,AN,9.0
1,AP,3220.0
2,AR,87.0
3,AS,1499.0
4,BR,30.0


## Getting State and Delta delta7

**Created Function to get Extracted Data With Two Column for.eg State and Delta**

In [50]:
def to_dataframe(column1,column2):
    vv=np.array(tr.loc[:,[column1,column2]])
    DI={}
    for i in range(len(vv)):
        DI[vv[i][0]]=vv[i][1]
    column2=pd.DataFrame(DI).transpose()
    column2.insert(0,"State",FIRST_COL)
    column2.reset_index(drop=True, inplace=True)
    return column2

In [51]:
#Getting State and Delta7
delta7=to_dataframe(column1="State",column2="delta7")
delta7.head(5)

,State,confirmed,recovered,tested,vaccinated1,vaccinated2,deceased,other
0,AN,3.0,5.0,8936.0,884.0,10640.0,NaN,NaN
1,AP,2873.0,3590.0,254532.0,1223010.0,1887005.0,30.0,NaN
2,AR,66.0,97.0,4788.0,3312.0,23647.0,NaN,NaN
3,AS,2056.0,2215.0,269097.0,274869.0,849889.0,24.0,NaN
4,BR,40.0,31.0,1378539.0,1286708.0,2144970.0,NaN,NaN


In [52]:
# changing names of columns
delta7.columns=["State","delta7_confirmed","delta7_recovered","delta7_tested","delta7_vaccinated1","delta7_vaccinated2","delta7_deceased","delta7_other"]
delta7.head(5)

,State,delta7_confirmed,delta7_recovered,delta7_tested,delta7_vaccinated1,delta7_vaccinated2,delta7_deceased,delta7_other
0,AN,3.0,5.0,8936.0,884.0,10640.0,NaN,NaN
1,AP,2873.0,3590.0,254532.0,1223010.0,1887005.0,30.0,NaN
2,AR,66.0,97.0,4788.0,3312.0,23647.0,NaN,NaN
3,AS,2056.0,2215.0,269097.0,274869.0,849889.0,24.0,NaN
4,BR,40.0,31.0,1378539.0,1286708.0,2144970.0,NaN,NaN


## Extracting State Total

In [53]:
#getting State And total
total=to_dataframe(column1="State",column2="total")
# Changing Column Names of obtained Dataset
total.columns
li= ['State','total_confirmed','total_deceased','total_recovered','total_tested','total_vaccinated1','total_vaccinated2','total_other']
total.columns=li
total.head(5)

,State,total_confirmed,total_deceased,total_recovered,total_tested,total_vaccinated1,total_vaccinated2,total_other
0,AN,7651.0,129.0,7518.0,598033.0,294001.0,200157.0,NaN
1,AP,2066450.0,14373.0,2047722.0,29518787.0,32976969.0,20375181.0,NaN
2,AR,55155.0,280.0,54774.0,1185436.0,771875.0,534486.0,NaN
3,AS,610645.0,5997.0,600974.0,24712042.0,20172463.0,8068795.0,1347.0
4,BR,726098.0,9661.0,716390.0,50531824.0,49874828.0,18346781.0,1.0


## Extracing Delta7

In [54]:
# merging two Datasets
del_del21=deltaa.merge(delta21_14,on="State")
del_del21_del7=del_del21.merge(delta7,on="State")
del_del21_del7_total = del_del21_del7.merge(total,on="State")
del_del21_del7_total

,State,tested,vaccinated1,vaccinated2,confirmed,deceased,recovered,other,delta21_14_confirmed,delta7_confirmed,...,delta7_vaccinated2,delta7_deceased,delta7_other,total_confirmed,total_deceased,total_recovered,total_tested,total_vaccinated1,total_vaccinated2,total_other
0,AN,1376.0,3.0,13.0,NaN,NaN,NaN,NaN,9.0,3.0,...,10640.0,NaN,NaN,7651.0,129.0,7518.0,598033.0,294001.0,200157.0,NaN
1,AP,39848.0,20497.0,24137.0,385.0,4.0,675.0,NaN,3220.0,2873.0,...,1887005.0,30.0,NaN,2066450.0,14373.0,2047722.0,29518787.0,32976969.0,20375181.0,NaN
2,AR,334.0,42.0,195.0,1.0,NaN,9.0,NaN,87.0,66.0,...,23647.0,NaN,NaN,55155.0,280.0,54774.0,1185436.0,771875.0,534486.0,NaN
3,AS,15060.0,19124.0,37463.0,212.0,1.0,236.0,NaN,1499.0,2056.0,...,849889.0,24.0,NaN,610645.0,5997.0,600974.0,24712042.0,20172463.0,8068795.0,1347.0
4,BR,226443.0,114694.0,145827.0,8.0,NaN,9.0,NaN,30.0,40.0,...,2144970.0,NaN,NaN,726098.0,9661.0,716390.0,50531824.0,49874828.0,18346781.0,1.0
5,CH,1403.0,211.0,1282.0,5.0,NaN,3.0,NaN,23.0,28.0,...,21641.0,NaN,NaN,65351.0,820.0,64495.0,792851.0,926035.0,546981.0,NaN
6,CT,11869.0,21312.0,39393.0,32.0,1.0,32.0,NaN,124.0,205.0,...,604260.0,5.0,NaN,1006052.0,13577.0,992159.0,13709510.0,14851682.0,7343273.0,NaN
7,DL,56751.0,12482.0,11839.0,45.0,NaN,46.0,NaN,195.0,267.0,...,269146.0,NaN,NaN,1439870.0,25091.0,1414431.0,29427753.0,13055636.0,7425404.0,NaN
8,DN,NaN,3.0,20.0,NaN,NaN,1.0,NaN,4.0,NaN,...,14244.0,NaN,NaN,10681.0,4.0,10644.0,72410.0,660753.0,370255.0,31.0
9,GA,2361.0,2572.0,12404.0,23.0,NaN,53.0,NaN,409.0,222.0,...,46494.0,6.0,NaN,178108.0,3364.0,174392.0,1468399.0,1262568.0,911114.0,NaN


In [55]:
metaa=to_dataframe(column1="State",column2="meta")

In [38]:
# def to_dataframe(column1,column2):
#     vv=np.array(tr.loc[:,[column1,column2]])
#     DI={}
#     for i in range(len(vv)):
#         DI[vv[i][0]]=vv[i][1]
#     column2=pd.DataFrame(DI).transpose()
#     column2.insert(0,"State",FIRST_COL)
#     column2.reset_index(drop=True, inplace=True)
#     return column2

In [56]:
metaa.head(5)

,State,date,last_updated,population,tested,vaccinated,notes
0,AN,2021-10-31,2021-11-01T11:03:10+05:30,397000,"{'date': '2021-10-31', 'source': 'https://dhs....",{'date': '2021-10-31'},NaN
1,AP,2021-10-31,2021-11-01T09:54:14+05:30,52221000,"{'date': '2021-10-31', 'source': 'https://twit...",{'date': '2021-10-31'},NaN
2,AR,2021-10-31,2021-11-01T11:20:24+05:30,1504000,"{'date': '2021-10-31', 'source': 'https://twit...",{'date': '2021-10-31'},[July 25]: All numbers corresponding to Papum ...
3,AS,2021-10-31,2021-11-01T09:54:14+05:30,34293000,"{'date': '2021-10-31', 'source': 'https://twit...",{'date': '2021-10-31'},[Jan 1]: 1347 cases i.e Covid +'tive patients ...
4,BR,2021-10-31,2021-11-01T09:54:14+05:30,119520000,"{'date': '2021-10-31', 'source': 'https://twit...",{'date': '2021-10-31'},[June 9] : 3951 deceased cases have been repor...


In [57]:
#getting meta values
vv=np.array(metaa.loc[:,["State","population"]])
#getting Population
polpulation=pd.DataFrame(vv)
polpulation.columns=["State","Polpulation"]
del_del21_del7_total_population=del_del21_del7_total.merge(polpulation,on="State")
del_del21_del7_total_population.head(5)


,State,tested,vaccinated1,vaccinated2,confirmed,deceased,recovered,other,delta21_14_confirmed,delta7_confirmed,...,delta7_deceased,delta7_other,total_confirmed,total_deceased,total_recovered,total_tested,total_vaccinated1,total_vaccinated2,total_other,Polpulation
0,AN,1376.0,3.0,13.0,NaN,NaN,NaN,NaN,9.0,3.0,...,NaN,NaN,7651.0,129.0,7518.0,598033.0,294001.0,200157.0,NaN,397000
1,AP,39848.0,20497.0,24137.0,385.0,4.0,675.0,NaN,3220.0,2873.0,...,30.0,NaN,2066450.0,14373.0,2047722.0,29518787.0,32976969.0,20375181.0,NaN,52221000
2,AR,334.0,42.0,195.0,1.0,NaN,9.0,NaN,87.0,66.0,...,NaN,NaN,55155.0,280.0,54774.0,1185436.0,771875.0,534486.0,NaN,1504000
3,AS,15060.0,19124.0,37463.0,212.0,1.0,236.0,NaN,1499.0,2056.0,...,24.0,NaN,610645.0,5997.0,600974.0,24712042.0,20172463.0,8068795.0,1347.0,34293000
4,BR,226443.0,114694.0,145827.0,8.0,NaN,9.0,NaN,30.0,40.0,...,NaN,NaN,726098.0,9661.0,716390.0,50531824.0,49874828.0,18346781.0,1.0,119520000


In [58]:
population_column =del_del21_del7_total_population.pop("Polpulation")
del_del21_del7_total_population.insert(1,"Population",population_column)

In [59]:
dates=metaa.loc[:,["State","date","last_updated"]]

In [60]:
del_del21_del7_total_population_dates=del_del21_del7_total_population.merge(dates,on="State")

In [61]:
def editing_columns(value_want_to_replace,place_at,data_set,column_name_you_want_to_give):
    xx =data_set.pop(value_want_to_replace)
    yy=data_set.insert(place_at,column_name_you_want_to_give,xx)
    return yy

In [62]:
del_del21_del7_total_population_dates.head(5)

,State,Population,tested,vaccinated1,vaccinated2,confirmed,deceased,recovered,other,delta21_14_confirmed,...,delta7_other,total_confirmed,total_deceased,total_recovered,total_tested,total_vaccinated1,total_vaccinated2,total_other,date,last_updated
0,AN,397000,1376.0,3.0,13.0,NaN,NaN,NaN,NaN,9.0,...,NaN,7651.0,129.0,7518.0,598033.0,294001.0,200157.0,NaN,2021-10-31,2021-11-01T11:03:10+05:30
1,AP,52221000,39848.0,20497.0,24137.0,385.0,4.0,675.0,NaN,3220.0,...,NaN,2066450.0,14373.0,2047722.0,29518787.0,32976969.0,20375181.0,NaN,2021-10-31,2021-11-01T09:54:14+05:30
2,AR,1504000,334.0,42.0,195.0,1.0,NaN,9.0,NaN,87.0,...,NaN,55155.0,280.0,54774.0,1185436.0,771875.0,534486.0,NaN,2021-10-31,2021-11-01T11:20:24+05:30
3,AS,34293000,15060.0,19124.0,37463.0,212.0,1.0,236.0,NaN,1499.0,...,NaN,610645.0,5997.0,600974.0,24712042.0,20172463.0,8068795.0,1347.0,2021-10-31,2021-11-01T09:54:14+05:30
4,BR,119520000,226443.0,114694.0,145827.0,8.0,NaN,9.0,NaN,30.0,...,NaN,726098.0,9661.0,716390.0,50531824.0,49874828.0,18346781.0,1.0,2021-10-31,2021-11-01T09:54:14+05:30


In [63]:
editing_columns(value_want_to_replace="date",place_at=2,data_set=del_del21_del7_total_population_dates,column_name_you_want_to_give="Date")

In [64]:
editing_columns(value_want_to_replace="last_updated",place_at=3,data_set=del_del21_del7_total_population_dates,column_name_you_want_to_give="Last_Update",)

In [65]:
del_del21_del7_total_population_dates.head(5)

,State,Population,Date,Last_Update,tested,vaccinated1,vaccinated2,confirmed,deceased,recovered,...,delta7_vaccinated2,delta7_deceased,delta7_other,total_confirmed,total_deceased,total_recovered,total_tested,total_vaccinated1,total_vaccinated2,total_other
0,AN,397000,2021-10-31,2021-11-01T11:03:10+05:30,1376.0,3.0,13.0,NaN,NaN,NaN,...,10640.0,NaN,NaN,7651.0,129.0,7518.0,598033.0,294001.0,200157.0,NaN
1,AP,52221000,2021-10-31,2021-11-01T09:54:14+05:30,39848.0,20497.0,24137.0,385.0,4.0,675.0,...,1887005.0,30.0,NaN,2066450.0,14373.0,2047722.0,29518787.0,32976969.0,20375181.0,NaN
2,AR,1504000,2021-10-31,2021-11-01T11:20:24+05:30,334.0,42.0,195.0,1.0,NaN,9.0,...,23647.0,NaN,NaN,55155.0,280.0,54774.0,1185436.0,771875.0,534486.0,NaN
3,AS,34293000,2021-10-31,2021-11-01T09:54:14+05:30,15060.0,19124.0,37463.0,212.0,1.0,236.0,...,849889.0,24.0,NaN,610645.0,5997.0,600974.0,24712042.0,20172463.0,8068795.0,1347.0
4,BR,119520000,2021-10-31,2021-11-01T09:54:14+05:30,226443.0,114694.0,145827.0,8.0,NaN,9.0,...,2144970.0,NaN,NaN,726098.0,9661.0,716390.0,50531824.0,49874828.0,18346781.0,1.0


In [66]:
notes = metaa.loc[:,["State","notes"]]

In [67]:
notes.columns

Index(['State', 'notes'], dtype='object')

In [68]:
#del_del21_del7_total_population_dates.to_csv(r"C:\Users\91940\Desktop\Python Project\sample2.csv",index=False)

In [69]:
#to_dataframe(column1="State",column2="districts").transpose()

## Manupulation For District

 
 ...............................

In [70]:
new_data=tr.drop("TT",axis=0)
district=new_data.loc[:,["State","districts"]]
district=np.array(district)
x=pd.DataFrame(district)

In [71]:
diii={}
for i in range(int(len(district))):
    value=list(dict(np.array(x)[i][1]).keys())
    key=(np.array(x)[i][0])
    diii[key]=value

In [86]:
stat=[]
district=[]
for i,j in diii.items():
    for k in range(len(j)):
        stat.append(i)
        district.append(j[k])

In [87]:
dis=pd.Series(district)
st=pd.Series(stat)
l={}
State_district=pd.DataFrame(l)

In [88]:
State_district.insert(0,"State",st)
State_district.insert(1,"District",dis)
State_district

,State,District
0,AN,Nicobars
1,AN,North and Middle Andaman
2,AN,South Andaman
3,AN,Unknown
4,AP,Anantapur
...,...,...
743,WB,Purba Bardhaman
744,WB,Purba Medinipur
745,WB,Purulia
746,WB,South 24 Parganas


In [64]:
#State_district.to_csv(r"C:\Users\91940\Desktop\Python Project\dist1.csv",index=False,header=True)

## I want District-And other data now

In [89]:
pd.DataFrame(new_data.loc[:,"districts"][0]).transpose()

,delta7,meta,total,delta,delta21_14
Nicobars,"{'vaccinated1': 62, 'vaccinated2': 811}",{'population': 36842},"{'vaccinated1': 25394, 'vaccinated2': 20313}",NaN,NaN
North and Middle Andaman,"{'vaccinated1': 90, 'vaccinated2': 1839}",{'population': 105597},"{'vaccinated1': 78945, 'vaccinated2': 59522}",{'vaccinated2': 8},NaN
South Andaman,"{'vaccinated1': 732, 'vaccinated2': 8012}",{'population': 238142},"{'vaccinated1': 189662, 'vaccinated2': 120322}","{'vaccinated1': 3, 'vaccinated2': 28}",NaN
Unknown,"{'confirmed': 3, 'recovered': 5}",NaN,"{'confirmed': 7651, 'deceased': 129, 'recovere...",NaN,{'confirmed': 9}


In [90]:
Concat_values=pd.DataFrame(new_data.loc[:,"districts"][0]).transpose()
for i in range(1,len(new_data.loc[:,"districts"])):
    first_st=pd.DataFrame(new_data.loc[:,"districts"][i]).transpose()
    Concat_values=pd.concat([Concat_values,first_st])

In [91]:
Concat_values.index

Index(['Nicobars', 'North and Middle Andaman', 'South Andaman', 'Unknown',
       'Anantapur', 'Chittoor', 'East Godavari', 'Foreign Evacuees', 'Guntur',
       'Krishna',
       ...
       'Nadia', 'North 24 Parganas', 'Other State', 'Paschim Bardhaman',
       'Paschim Medinipur', 'Purba Bardhaman', 'Purba Medinipur', 'Purulia',
       'South 24 Parganas', 'Uttar Dinajpur'],
      dtype='object', length=748)

In [92]:
col1 = np.array(Concat_values.index)

In [93]:
Concat_values.insert(0,"District",col1)
Concat_values.head(5)

,District,delta7,meta,total,delta,delta21_14
Nicobars,Nicobars,"{'vaccinated1': 62, 'vaccinated2': 811}",{'population': 36842},"{'vaccinated1': 25394, 'vaccinated2': 20313}",NaN,NaN
North and Middle Andaman,North and Middle Andaman,"{'vaccinated1': 90, 'vaccinated2': 1839}",{'population': 105597},"{'vaccinated1': 78945, 'vaccinated2': 59522}",{'vaccinated2': 8},NaN
South Andaman,South Andaman,"{'vaccinated1': 732, 'vaccinated2': 8012}",{'population': 238142},"{'vaccinated1': 189662, 'vaccinated2': 120322}","{'vaccinated1': 3, 'vaccinated2': 28}",NaN
Unknown,Unknown,"{'confirmed': 3, 'recovered': 5}",NaN,"{'confirmed': 7651, 'deceased': 129, 'recovere...",NaN,{'confirmed': 9}
Anantapur,Anantapur,"{'confirmed': 38, 'recovered': 58, 'vaccinated...","{'population': 4083315, 'tested': {'date': '20...","{'confirmed': 157843, 'deceased': 1093, 'recov...","{'confirmed': 4, 'recovered': 5, 'vaccinated1'...",{'confirmed': 50}


In [94]:
vv=np.array(Concat_values.loc[:,["District","delta"]])
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]
n=pd.DataFrame(DI).transpose()
co=np.array(n.index)
n.insert(0,"District",co)
ll=['District', 'delta_vaccinated2', 'delta_vaccinated1', 'delta_confirmed', 'delta_deceased',
       'delta_recovered', 'delta_tested', 'delta_other']
n.columns=ll
n.reset_index(drop=True, inplace=True)
state_delta=State_district.merge(n,on="District")

In [95]:
def to_dataframe(column1,column2):
    vv=np.array(Concat_values.loc[:,[column1,column2]])
    DI={}
    for i in range(len(vv)):
        DI[vv[i][0]]=vv[i][1]
    column2=pd.DataFrame(DI).transpose()
    co=np.array(column2.index)
    column2.insert(0,"District",co)
    column2.reset_index(drop=True, inplace=True)
    return column2

In [96]:
n=to_dataframe(column1="District",column2="delta")
n.columns

Index(['District', 'vaccinated2', 'vaccinated1', 'confirmed', 'deceased',
       'recovered', 'tested', 'other'],
      dtype='object')

In [97]:
ll=['District', 'delta_vaccinated2', 'delta_vaccinated1', 'delta_confirmed', 'delta_deceased',
       'delta_recovered', 'delta_tested', 'delta_other']
n.columns=ll

In [98]:
State_district.head(5)

,State,District
0,AN,Nicobars
1,AN,North and Middle Andaman
2,AN,South Andaman
3,AN,Unknown
4,AP,Anantapur


In [99]:
state_delta=State_district.merge(n,on="District")

In [100]:
state_delta.head(5)

,State,District,delta_vaccinated2,delta_vaccinated1,delta_confirmed,delta_deceased,delta_recovered,delta_tested,delta_other
0,AN,Nicobars,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AN,North and Middle Andaman,8.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AN,South Andaman,28.0,3.0,NaN,NaN,NaN,NaN,NaN
3,AN,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN
4,AS,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN


In [101]:
n=to_dataframe(column1="District",column2="delta7")
n.columns
ll=['District','delta7_vaccinated1','delta7_vaccinated2','delta7_confirmed','delta7_deceased','delta7_recovered','delta7_tested','delta7_other']
n.columns=ll

In [102]:
state_delta_delta7=state_delta.merge(n,on="District")

In [103]:
state_delta_delta7.head(5)

,State,District,delta_vaccinated2,delta_vaccinated1,delta_confirmed,delta_deceased,delta_recovered,delta_tested,delta_other,delta7_vaccinated1,delta7_vaccinated2,delta7_confirmed,delta7_deceased,delta7_recovered,delta7_tested,delta7_other
0,AN,Nicobars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,811.0,NaN,NaN,NaN,NaN,NaN
1,AN,North and Middle Andaman,8.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,1839.0,NaN,NaN,NaN,NaN,NaN
2,AN,South Andaman,28.0,3.0,NaN,NaN,NaN,NaN,NaN,732.0,8012.0,NaN,NaN,NaN,NaN,NaN
3,AN,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,NaN,1189.0,9.0,1121.0,NaN,NaN
4,AS,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,NaN,1189.0,9.0,1121.0,NaN,NaN


In [104]:
n=to_dataframe(column1="District",column2="delta21_14")
n.columns
ll=['District', 'delta21_14_confirmed']
n.columns=ll
state_delta_delta7_delta21_14=state_delta_delta7.merge(n,on="District")
state_delta_delta7_delta21_14.head(5)

,State,District,delta_vaccinated2,delta_vaccinated1,delta_confirmed,delta_deceased,delta_recovered,delta_tested,delta_other,delta7_vaccinated1,delta7_vaccinated2,delta7_confirmed,delta7_deceased,delta7_recovered,delta7_tested,delta7_other,delta21_14_confirmed
0,AN,Nicobars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,811.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AN,North and Middle Andaman,8.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,1839.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AN,South Andaman,28.0,3.0,NaN,NaN,NaN,NaN,NaN,732.0,8012.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AN,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,NaN,1189.0,9.0,1121.0,NaN,NaN,1068.0
4,AS,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,NaN,1189.0,9.0,1121.0,NaN,NaN,1068.0


In [106]:
n=to_dataframe(column1="District",column2="total")
n.columns
ll=['District', 'total_vaccinated1', 'total_vaccinated2', 'total_confirmed', 'total_deceased',
       'total_recovered', 'total_tested', 'total_other']
n.columns=ll
state_delta_delta7_delta21_14_total=state_delta_delta7_delta21_14.merge(n,on="District")
state_delta_delta7_delta21_14_total.head(5)

,State,District,delta_vaccinated2,delta_vaccinated1,delta_confirmed,delta_deceased,delta_recovered,delta_tested,delta_other,delta7_vaccinated1,...,delta7_tested,delta7_other,delta21_14_confirmed,total_vaccinated1,total_vaccinated2,total_confirmed,total_deceased,total_recovered,total_tested,total_other
0,AN,Nicobars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,...,NaN,NaN,NaN,25394.0,20313.0,NaN,NaN,NaN,NaN,NaN
1,AN,North and Middle Andaman,8.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,...,NaN,NaN,NaN,78945.0,59522.0,NaN,NaN,NaN,NaN,NaN
2,AN,South Andaman,28.0,3.0,NaN,NaN,NaN,NaN,NaN,732.0,...,NaN,NaN,NaN,189662.0,120322.0,NaN,NaN,NaN,NaN,NaN
3,AN,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,...,NaN,NaN,1068.0,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN
4,AS,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,...,NaN,NaN,1068.0,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN


In [107]:
n=to_dataframe(column1="District",column2="meta")
n=n.iloc[:,0:4]
n.columns
ll=['District', 'Population', 'meta_tested', 'meta_vaccinated']
n.columns=ll
complete_district=state_delta_delta7_delta21_14_total.merge(n,on="District")
complete_district.head(5)

,State,District,delta_vaccinated2,delta_vaccinated1,delta_confirmed,delta_deceased,delta_recovered,delta_tested,delta_other,delta7_vaccinated1,...,total_vaccinated1,total_vaccinated2,total_confirmed,total_deceased,total_recovered,total_tested,total_other,Population,meta_tested,meta_vaccinated
0,AN,Nicobars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,...,25394.0,20313.0,NaN,NaN,NaN,NaN,NaN,36842.0,NaN,NaN
1,AN,North and Middle Andaman,8.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,...,78945.0,59522.0,NaN,NaN,NaN,NaN,NaN,105597.0,NaN,NaN
2,AN,South Andaman,28.0,3.0,NaN,NaN,NaN,NaN,NaN,732.0,...,189662.0,120322.0,NaN,NaN,NaN,NaN,NaN,238142.0,NaN,NaN
3,AN,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,...,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN,NaN,NaN,NaN
4,AS,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,...,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN,NaN,NaN,NaN


In [108]:
complete_district.head(5)

,State,District,delta_vaccinated2,delta_vaccinated1,delta_confirmed,delta_deceased,delta_recovered,delta_tested,delta_other,delta7_vaccinated1,...,total_vaccinated1,total_vaccinated2,total_confirmed,total_deceased,total_recovered,total_tested,total_other,Population,meta_tested,meta_vaccinated
0,AN,Nicobars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,...,25394.0,20313.0,NaN,NaN,NaN,NaN,NaN,36842.0,NaN,NaN
1,AN,North and Middle Andaman,8.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,...,78945.0,59522.0,NaN,NaN,NaN,NaN,NaN,105597.0,NaN,NaN
2,AN,South Andaman,28.0,3.0,NaN,NaN,NaN,NaN,NaN,732.0,...,189662.0,120322.0,NaN,NaN,NaN,NaN,NaN,238142.0,NaN,NaN
3,AN,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,...,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN,NaN,NaN,NaN
4,AS,Unknown,NaN,NaN,121.0,1.0,183.0,NaN,NaN,NaN,...,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN,NaN,NaN,NaN


In [109]:
po=complete_district.pop("Population")
complete_district.insert(2,"Population",po)
complete_district.head(5)

,State,District,Population,delta_vaccinated2,delta_vaccinated1,delta_confirmed,delta_deceased,delta_recovered,delta_tested,delta_other,...,delta21_14_confirmed,total_vaccinated1,total_vaccinated2,total_confirmed,total_deceased,total_recovered,total_tested,total_other,meta_tested,meta_vaccinated
0,AN,Nicobars,36842.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25394.0,20313.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AN,North and Middle Andaman,105597.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,78945.0,59522.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AN,South Andaman,238142.0,28.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,189662.0,120322.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AN,Unknown,NaN,NaN,NaN,121.0,1.0,183.0,NaN,NaN,...,1068.0,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN,NaN,NaN
4,AS,Unknown,NaN,NaN,NaN,121.0,1.0,183.0,NaN,NaN,...,1068.0,NaN,NaN,671463.0,3956.0,663498.0,NaN,NaN,NaN,NaN


In [109]:
#complete_district.to_csv(r"C:\Users\91940\Desktop\Python Project\district.csv",index=False)

In [110]:
complete_district.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 747
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   State                 748 non-null    object 
 1   District              748 non-null    object 
 2   Population            709 non-null    object 
 3   delta_vaccinated2     656 non-null    float64
 4   delta_vaccinated1     645 non-null    float64
 5   delta_confirmed       269 non-null    float64
 6   delta_deceased        60 non-null     float64
 7   delta_recovered       264 non-null    float64
 8   delta_tested          3 non-null      float64
 9   delta_other           1 non-null      float64
 10  delta7_vaccinated1    718 non-null    float64
 11  delta7_vaccinated2    718 non-null    float64
 12  delta7_confirmed      419 non-null    float64
 13  delta7_deceased       148 non-null    float64
 14  delta7_recovered      404 non-null    float64
 15  delta7_tested         3